# CounterGAN Tabular Fine-Tune (German Credit)

This notebook implements:
1. Clustering of users into profiles.
2. Loading of cluster-specific immutable features & cost weights.
3. Personalized CounterGAN training per cluster.
4. Generation and evaluation of counterfactuals (baseline vs personalized).
5. Hyperparameter tuning (randomized grid search).
6. Saving the final recommended profile configuration.


In [1]:

import os
import time
import json
import random
import itertools
import jsonpickle


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
tf.compat.v1.enable_eager_execution()   # TF1-style eager mode

from datetime import datetime
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import defaultdict

from tensorflow.keras import Model, optimizers
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Add, Input, ActivityRegularization, Dropout
# Alibi logger
import logging
alibi_logger = logging.getLogger('alibi')
alibi_logger.setLevel('INFO')

BASE_PATH = "./counterfactuals"

if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)


date = datetime.now().strftime('%Y-%m-%d')
EXPERIMENT_PATH = f"{BASE_PATH}/diabetes_{date}"
MODELS_EXPERIMENT_PATH = f"{BASE_PATH}/diabetes_2020-09-09"
if not os.path.exists(EXPERIMENT_PATH):
    os.makedirs(EXPERIMENT_PATH)
    
# )

2025-07-10 10:55:05.362571: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 10:55:05.407449: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-10 10:55:05.407482: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-10 10:55:05.407518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-10 10:55:05.417068: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 10:55:05.417749: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [29]:
INITIAL_CLASS = 0
DESIRED_CLASS = 1
N_CLASSES = 2
# n_training_iterations = 10

# Set random seeds for reproducibility
np.set_printoptions(precision=2)
random.seed(2020)
np.random.seed(2020)
tf.random.set_seed(2020)

# German Credit dataset

def preprocess_data_german(df, target_column="Outcome"):
    """
    Preprocess the German Credit dataset by encoding categorical variables and splitting the data into 
    train, test, and user simulation sets.
    
    Returns a dictionary with processed train, test, and user datasets.
    """
    
    # Assign meaningful column names
    df.columns = [
        'Status', 'Month', 'Credit_History', 'Purpose', 'Credit_Amount',
        'Savings', 'Employment', 'Installment_Rate', 'Personal_Status', 'Other_Debtors',
        'Residence_Duration', 'Property', 'Age', 'Other_Installment_Plans', 'Housing',
        'Existing_Credits', 'Job', 'Num_Liable_People', 'Telephone', 'Foreign_Worker',
        'Outcome'
    ]
    
    # Mapping categorical features to more meaningful values
    status_mapping = { 'A11': '< 0 DM', 'A12': '0 <= ... < 200 DM', 'A13': '>= 200 DM / salary assignments for at least 1 year', 'A14': 'no checking account' }
    credit_history_mapping = { 'A30': 'no credits taken/ all credits paid back duly', 'A31': 'all credits at this bank paid back duly', 'A32': 'existing credits paid back duly till now', 'A33': 'delay in paying off in the past', 'A34': 'critical account/other credits existing' }
    savings_mapping = { 'A61': '< 100 DM', 'A62': '100 <= ... < 500 DM', 'A63': '500 <= ... < 1000 DM', 'A64': '>= 1000 DM', 'A65': 'unknown/no savings account' }
    employment_mapping = { 'A71': 'unemployed', 'A72': '< 1 year', 'A73': '1 <= ... < 4 years', 'A74': '4 <= ... < 7 years', 'A75': '>= 7 years' }
    personal_status_mapping = { 'A91': 'male: divorced/separated', 'A92': 'female: divorced/separated/married', 'A93': 'male: single', 'A94': 'male: married/widowed', 'A95': 'female: single' }
    other_debtors_mapping = { 'A101': 'none', 'A102': 'co-applicant', 'A103': 'guarantor' }
    property_mapping = { 'A121': 'real estate', 'A122': 'building society savings agreement/life insurance', 'A123': 'car or other, not in attribute 6', 'A124': 'unknown/no property' }
    other_installment_plans_mapping = { 'A141': 'bank', 'A142': 'stores', 'A143': 'none' }
    housing_mapping = { 'A151': 'rent', 'A152': 'own', 'A153': 'for free' }
    telephone_mapping = { 'A191': 'none', 'A192': 'yes, registered under the customer\'s name' }
    foreign_worker_mapping = { 'A201': 'yes', 'A202': 'no' }

    # Apply mappings
    df['Status'] = df['Status'].map(status_mapping)
    df['Credit_History'] = df['Credit_History'].map(credit_history_mapping)
    df['Savings'] = df['Savings'].map(savings_mapping)
    df['Employment'] = df['Employment'].map(employment_mapping)
    df['Personal_Status'] = df['Personal_Status'].map(personal_status_mapping)
    df['Other_Debtors'] = df['Other_Debtors'].map(other_debtors_mapping)
    df['Property'] = df['Property'].map(property_mapping)
    df['Other_Installment_Plans'] = df['Other_Installment_Plans'].map(other_installment_plans_mapping)
    df['Housing'] = df['Housing'].map(housing_mapping)
    df['Telephone'] = df['Telephone'].map(telephone_mapping)
    df['Foreign_Worker'] = df['Foreign_Worker'].map(foreign_worker_mapping)
    df_pre_encoding = df.copy()

    # Encode ordinal columns
    ordinal_cols = ['Status', 'Credit_History', 'Savings', 'Employment']
    le = LabelEncoder()
    for col in ordinal_cols:
        df[col] = le.fit_transform(df[col])

    # One-hot encode nominal columns
    nominal_columns = ['Purpose', 'Personal_Status', 'Other_Debtors', 'Property', 
                       'Other_Installment_Plans', 'Housing', 'Job', 'Telephone', 'Foreign_Worker']
    df = pd.get_dummies(df, columns=nominal_columns, drop_first=True)

    # Process target variable
    Y = df[target_column].replace(1, 0).replace(2, 1)
    X = df.drop(columns=[target_column])

    # Get final feature set
    # list all features
    # immutable_features = set(X.columns) - set(['Status', 'Credit_History'])
    

    # mutable_features = set(X.columns) - set(immutable_features)
    # mutable_features = list(mutable_features)

    features = list(set(X.columns))

    return  X, Y, features, df_pre_encoding
    
# =========================================================


# Make sure 'german.csv' is in your project directory
df = pd.read_csv('statlog_german_credit_data/german.data', sep=' ', skiprows=1, header=None)
# 1) Run your existing preprocessing
X, Y, features, df_pre_encoding = preprocess_data_german(df)

# 2) Grab the list of encoded column names (in order) from X
encoded_cols = list(X.columns)

# 3) Get your “raw” feature names, i.e. the columns in df_pre_encoding before one-hot
raw_cols = [c for c in df_pre_encoding.columns if c != "Outcome"]

# 4) Build the mapping raw_feature → list of encoded indices
raw_to_encoded = {}
for raw in raw_cols:
    # any encoded column exactly equal (for your ordinal cols)
    # or starting with “raw_” (for one-hot dummies)
    idxs = [
        i for i, c in enumerate(encoded_cols)
        if c == raw or c.startswith(raw + "_")
    ]
    raw_to_encoded[raw] = idxs

# 5) (Optional) Inspect
for raw, idxs in raw_to_encoded.items():
    print(f"{raw:20s} → {len(idxs):2d} cols @ indices {idxs[:5]}{'…' if len(idxs)>5 else ''}")


X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=2020)

standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



Status               →  1 cols @ indices [0]
Month                →  1 cols @ indices [1]
Credit_History       →  1 cols @ indices [2]
Purpose              →  9 cols @ indices [11, 12, 13, 14, 15]…
Credit_Amount        →  1 cols @ indices [3]
Savings              →  1 cols @ indices [4]
Employment           →  1 cols @ indices [5]
Installment_Rate     →  1 cols @ indices [6]
Personal_Status      →  3 cols @ indices [20, 21, 22]
Other_Debtors        →  2 cols @ indices [23, 24]
Residence_Duration   →  1 cols @ indices [7]
Property             →  3 cols @ indices [25, 26, 27]
Age                  →  1 cols @ indices [8]
Other_Installment_Plans →  2 cols @ indices [28, 29]
Housing              →  2 cols @ indices [30, 31]
Existing_Credits     →  1 cols @ indices [9]
Job                  →  3 cols @ indices [32, 33, 34]
Num_Liable_People    →  1 cols @ indices [10]
Telephone            →  1 cols @ indices [35]
Foreign_Worker       →  1 cols @ indices [36]


In [14]:
# assume encoded_cols and raw_to_encoded are already defined from the previous step

for raw, idxs in raw_to_encoded.items():
    print(f"{raw:20s} → {len(idxs):2d} encoded columns:")
    for i in idxs:
        print(f"    [{i:3d}] {encoded_cols[i]}")
    print()


Status               →  1 encoded columns:
    [  0] Status

Month                →  1 encoded columns:
    [  1] Month

Credit_History       →  1 encoded columns:
    [  2] Credit_History

Purpose              →  9 encoded columns:
    [ 11] Purpose_A41
    [ 12] Purpose_A410
    [ 13] Purpose_A42
    [ 14] Purpose_A43
    [ 15] Purpose_A44
    [ 16] Purpose_A45
    [ 17] Purpose_A46
    [ 18] Purpose_A48
    [ 19] Purpose_A49

Credit_Amount        →  1 encoded columns:
    [  3] Credit_Amount

Savings              →  1 encoded columns:
    [  4] Savings

Employment           →  1 encoded columns:
    [  5] Employment

Installment_Rate     →  1 encoded columns:
    [  6] Installment_Rate

Personal_Status      →  3 encoded columns:
    [ 20] Personal_Status_male: divorced/separated
    [ 21] Personal_Status_male: married/widowed
    [ 22] Personal_Status_male: single

Other_Debtors        →  2 encoded columns:
    [ 23] Other_Debtors_guarantor
    [ 24] Other_Debtors_none

Residence_Du

In [3]:

# Load the classifier model
filename = f"{EXPERIMENT_PATH}/classifier.keras"
classifier = load_model(filename)
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.trainable = False
print(f"Classifier loaded from {filename}") 

# Load the autoencoder model
filename = f"{EXPERIMENT_PATH}/autoencoder.keras" 
autoencoder = load_model(filename)
# Ensure the autoencoder is compiled with the same optimizer and loss function  
autoencoder.compile(optimizer='nadam', loss='mse')

print(f"Autoencoder loaded from {filename}")

Classifier loaded from ./counterfactuals/diabetes_2025-07-10/classifier.keras
Autoencoder loaded from ./counterfactuals/diabetes_2025-07-10/autoencoder.keras


In [22]:
def compute_reconstruction_error(x, autoencoder):
    """Compute ∞-norm reconstruction error via autoencoder."""
    preds = autoencoder.predict(x)
    preds_flat = preds.reshape((preds.shape[0], -1))
    x_flat = x.reshape((x.shape[0], -1))
    # ∞-norm per sample, then average
    return np.mean(np.max(np.abs(preds_flat - x_flat), axis=1))

def print_training_information(generator, classifier, X_test, iteration, autoencoder):
    """Print diagnostic metrics during training."""
    X_gen = generator.predict(X_test)
    clf_test = classifier.predict(X_test)
    clf_gen  = classifier.predict(X_gen)
    delta_clf = (clf_gen - clf_test)[:, DESIRED_CLASS]

    recon_error = np.mean(compute_reconstruction_error(X_gen, autoencoder))
    l1_dist     = np.mean(np.abs(X_gen - X_test))

    print('='*88)
    print(f"Iteration {iteration} @ {datetime.now()}")
    print(f"Autoencoder reconstruction error (∞-norm): {recon_error:.3f}")
    print(f"Counterfactual prediction gain (0→1): {delta_clf.mean():.3f}")
    print(f"L1 distance (lower better): {l1_dist:.3f}")
    
def format_metric(metric):
    """Return a formatted version of a metric, with the confidence interval."""
    return f"{metric.mean():.3f} ± {1.96*metric.std()/np.sqrt(len(metric)):.3f}"

def compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None):
    """ Summarize the relevant metrics in a dictionary. """
    reconstruction_error = compute_reconstruction_error(counterfactuals, autoencoder)
    delta = np.abs(samples-counterfactuals)
    l1_distances = delta.reshape(delta.shape[0], -1).sum(axis=1)
    prediction_gain = (
        classifier.predict(counterfactuals)[:, DESIRED_CLASS] - 
        classifier.predict(samples)[:, DESIRED_CLASS]
    )

    metrics = dict()
    metrics["reconstruction_error"] = format_metric(np.array([reconstruction_error]))
    metrics["prediction_gain"] = format_metric(prediction_gain)
    metrics["sparsity"] = format_metric(l1_distances)
    metrics["latency"] = format_metric(latencies)
    batch_latency = batch_latency if batch_latency else sum(latencies)
    metrics["latency_batch"] = f"{batch_latency:.3f}"

    return metrics


def generate_fake_samples(x, generator):
    """Use the input generator to generate samples."""
    return generator.predict(x)

def data_stream(x, y=None, batch_size=500):
    """Generate batches until exhaustion of the input data."""
    n_train = x.shape[0]
    if y is not None:
        assert n_train == len(y)
    n_complete_batches, leftover = divmod(n_train, batch_size)
    n_batches = n_complete_batches + bool(leftover)

    perm = np.random.permutation(n_train)
    for i in range(n_batches):
        batch_idx = perm[i * batch_size:(i + 1) * batch_size]
        if y is not None:
            output = (x[batch_idx], y[batch_idx])
        else:
            output = x[batch_idx]
        yield output


def infinite_data_stream(x, y=None, batch_size=500):
    """Infinite batch generator."""
    batches = data_stream(x, y, batch_size=batch_size)
    while True:
        try:
            yield next(batches)
        except StopIteration:
            batches = data_stream(x, y, batch_size=batch_size)
            yield next(batches)

def create_generator(in_shape=(X_train.shape[1],), residuals=True):
    """Define and compile the residual generator of the CounteRGAN."""
    generator_input = Input(shape=in_shape, name='generator_input')
    generator = Dense(64, activation='relu')(generator_input)
    generator = Dense(32, activation='relu')(generator)
    generator = Dense(64, activation='relu')(generator)
    generator = Dense(in_shape[0], activation='tanh')(generator)
    generator_output = ActivityRegularization(l1=0., l2=1e-6)(generator)
    
    if residuals:
        generator_output = Add(name="output")([generator_input, generator_output])

    return Model(inputs=generator_input, outputs=generator_output)


def create_discriminator(in_shape=(X_train.shape[1],)):
    """ Define a neural network binary classifier to classify real and generated 
    examples."""
    model = Sequential([
        Dense(128, activation='relu', input_shape=in_shape),
        Dropout(0.2),
        Dense(1, activation='sigmoid'),
    ], name="discriminator")
    optimizer = optimizers.legacy.Adam(learning_rate=0.0005, beta_1=0.5, decay=1e-8)
    model.compile(optimizer, 'binary_crossentropy', ['accuracy'])
    return model


# def define_countergan(generator, discriminator, classifier, 
#                       input_shape=(X_train.shape[1],)):
#     """Combine a generator, discriminator, and fixed classifier into the CounteRGAN."""
#     discriminator.trainable = False
#     classifier.trainable = False

#     countergan_input = Input(shape=input_shape, name='countergan_input')
  
#     x_generated = generator(countergan_input)

#     countergan = Model(
#         inputs=countergan_input, 
#         outputs=[discriminator(x_generated), classifier(x_generated)]
#     )
        
#     optimizer = optimizers.legacy.RMSprop(learning_rate=2e-4, decay=1e-8)
#     countergan.compile(optimizer, ["binary_crossentropy", "categorical_crossentropy"])
#     return countergan

def define_countergan(generator, discriminator, classifier, cost_weights, sparsity_lambda=0.1,immutable_idxs=None):
    discriminator.trainable = False
    classifier.trainable    = False
    """
    Build the CounterGAN model that includes:
      - discriminator loss
      - classifier-target loss
      - PLUS a sparsity (L1) penalty on feature changes weighted by cost_weights
    """
    # inputs
    x_input = Input(shape=(X_train.shape[1],), name="cf_input")
    # generate counterfactual
    x_gen   = generator(x_input)
    # discriminator output
    y_disc  = discriminator(x_gen)
    # classifier output (for your desired class)
    y_clf   = classifier(x_gen)

    # compute the L1 distance between gen and original
    delta = x_gen - x_input  # shape: (batch, n_features)
    # convert cost_weights dict to a tensor of shape (n_features,)
    cost_vector = K.constant(
        [cost_weights.get(f, 1.0) for f in features], dtype="float32"
    ) 
    # per-sample L1 penalty = sum(|Δxᵢ| * cost_vectorᵢ)
    sparse_loss = K.sum(K.abs(delta) * cost_vector, axis=1)
    # define the full model
    cgan = tf.keras.models.Model(
        inputs=x_input, outputs=[y_disc, y_clf], name="countergan"
    )
    # add sparsity loss (scaled by lambda)
    cgan.add_loss(sparsity_lambda * K.mean(sparse_loss))
    # compile with existing GAN losses
    cgan.compile(
        optimizer=optimizers.legacy.RMSprop(learning_rate=2e-4, decay=1e-8),
        loss=["binary_crossentropy", "categorical_crossentropy"],
        loss_weights=[1.0, 1.0]
    )
    return cgan

def define_weighted_countergan(generator, discriminator, 
                               input_shape=(X_train.shape[1],)):
    """Combine a generator and a discriminator for the weighted version of the 
    CounteRGAN."""
    discriminator.trainable = False
    classifier.trainable = False
    countergan_input = Input(shape=input_shape, name='countergan_input')
  
    x_generated = generator(countergan_input)

    countergan = Model(inputs=countergan_input, outputs=discriminator(x_generated))
    optimizer = optimizers.legacy.RMSprop(learning_rate=5e-4, decay=1e-8)
    countergan.compile(optimizer, "binary_crossentropy")  
    return countergan

# def train_countergan(n_discriminator_steps, n_generator_steps, n_training_iterations,
#                      classifier, discriminator, generator, batches, 
#                      weighted_version=False):
#     """ Main function: train the CounteRGAN"""
#     def check_divergence(x_generated):
#         return np.all(np.isnan(x_generated))

#     def print_training_information(generator, classifier, X_test, iteration):
#         X_gen = generator.predict(X_test)
#         clf_pred_test = classifier.predict(X_test)
#         clf_pred = classifier.predict(X_gen)

#         delta_clf_pred = (clf_pred - clf_pred_test)[:, DESIRED_CLASS]
#         y_target = to_categorical([DESIRED_CLASS] * len(clf_pred), 
#                                   num_classes=N_CLASSES)
#         print('='*88)
#         print(f"Training iteration {iteration} at {datetime.now()}")
        
        
#         reconstruction_error = np.mean(compute_reconstruction_error(X_gen, autoencoder))
#         print(f"Autoencoder reconstruction error (infinity to 0): {reconstruction_error:.3f}")
#         print(f"Counterfactual prediction gain (0 to 1): {delta_clf_pred.mean():.3f}")
#         print(f"Sparsity (L1, infinity to 0): {np.mean(np.abs(X_gen-X_test)):.3f}")
#     cw = cluster_profile_map[cluster_id]["cost_weights"]
#     if weighted_version:
#         countergan = define_weighted_countergan(generator, discriminator)
#     else:
        
#         countergan = define_countergan(generator, discriminator, classifier,cost_weights=cw)

#     for iteration in range(n_training_iterations):
#         if iteration > 0:
#             x_generated = generator.predict(x_fake_input)
#             if check_divergence(x_generated):
#                 print("Training diverged with the following loss functions:")
#                 print(discrim_loss_1, discrim_accuracy, gan_loss, 
#                     discrim_loss, discrim_loss_2, clf_loss)
#                 break

#         # Periodically print and plot training information 
#         if (iteration % 1000 == 0) or (iteration == n_training_iterations - 1):
#             print_training_information(generator, classifier, X_test, iteration)

#         # Train the discriminator
#         discriminator.trainable = True
#         for _ in range(n_discriminator_steps):
#             x_fake_input, _ = next(batches)
#             x_fake = generate_fake_samples(x_fake_input, generator)
#             x_real = x_fake_input

#             x_batch = np.concatenate([x_real, x_fake])
#             y_batch = np.concatenate([np.ones(len(x_real)), np.zeros(len(x_fake))])
            
#             # Shuffle real and fake examples
#             p = np.random.permutation(len(y_batch))
#             x_batch, y_batch = x_batch[p], y_batch[p]

#             if weighted_version:
#                 classifier_scores = classifier.predict(x_batch)[:, DESIRED_CLASS]
                
#                 # The following update to the classifier scores is needed to have the 
#                 # same order of magnitude between real and generated samples losses
#                 real_samples = np.where(y_batch == 1.)
#                 average_score_real_samples = np.mean(classifier_scores[real_samples])
#                 classifier_scores[real_samples] /= average_score_real_samples
                
#                 fake_samples = np.where(y_batch == 0.)
#                 classifier_scores[fake_samples] = 1.

#                 discriminator.train_on_batch(
#                     x_batch, y_batch, sample_weight=classifier_scores
#                 )
#             else:
#                 discriminator.train_on_batch(x_batch, y_batch)

#         # Train the generator 
#         discriminator.trainable = False
#         for _ in range(n_generator_steps):
#             x_fake_input, _ = next(batches)
#             y_fake = np.ones(len(x_fake_input))
#             if weighted_version:
#                 countergan.train_on_batch(x_fake_input, y_fake)
#             else:
#                 y_target = to_categorical([DESIRED_CLASS] * len(x_fake_input), 
#                                           num_classes=N_CLASSES)
#                 countergan.train_on_batch(x_fake_input, [y_fake, y_target])
#     return countergan

def train_countergan(n_discriminator_steps,
                     n_generator_steps,
                     n_training_iterations,
                     classifier,
                     discriminator,
                     generator,
                     countergan,
                     batches,
                     weighted_version=False):
    """
    Train the CounterGAN model over provided batches.

    countergan: a compiled Keras Model with two outputs:
      [disc_output, class_output]
    """

    def check_divergence(x_generated):
        return np.all(np.isnan(x_generated))

    for iteration in range(n_training_iterations):
        # --- Discriminator updates ---
        discriminator.trainable = True
        for _ in range(n_discriminator_steps):
            x_real, _ = next(batches)
            x_fake = generator.predict(x_real)

            X_disc = np.vstack([x_real, x_fake])
            y_disc = np.concatenate([np.ones(len(x_real)), np.zeros(len(x_fake))])

            # shuffle
            idx = np.random.permutation(len(y_disc))
            X_disc, y_disc = X_disc[idx], y_disc[idx]

            if weighted_version:
                # weight real examples by classifier confidence
                clf_scores = classifier.predict(X_disc)[:, DESIRED_CLASS]
                real_mask = (y_disc == 1)
                avg_real = np.mean(clf_scores[real_mask]) + 1e-8
                clf_scores[real_mask] /= avg_real
                clf_scores[~real_mask] = 1.0
                discriminator.train_on_batch(X_disc, y_disc, sample_weight=clf_scores)
            else:
                discriminator.train_on_batch(X_disc, y_disc)

        # --- Generator (CounterGAN) updates ---
        discriminator.trainable = False
        for _ in range(n_generator_steps):
            x_in, _ = next(batches)
            # discriminator target: want them all judged as real
            y_fake = np.ones(len(x_in))
            # classifier target: push to desired class
            y_target = to_categorical([DESIRED_CLASS] * len(x_in), num_classes=N_CLASSES)

            if weighted_version:
                # reuse clf_scores to weight the combined update
                clf_scores = classifier.predict(x_in)[:, DESIRED_CLASS]
                # no need to normalize here; give equal weight
                countergan.train_on_batch(
                  x_in,
                  [y_fake, y_target],
                  sample_weight=clf_scores
                )
            else:
                countergan.train_on_batch(x_in, [y_fake, y_target])

        # optional divergence check
        if (iteration % 1000 == 0) or (iteration == n_training_iterations - 1):
            x_gen = generator.predict(x_real)
            if check_divergence(x_gen):
                print(f"⚠️  Divergence at iteration {iteration}, stopping early")
                break

    return countergan

In [5]:
import gower
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

# --- 0) raw_df: your un-encoded DataFrame of 37 columns, no target ---
raw_df = df_pre_encoding.drop(columns=['Outcome'])  # drop target column if present
# merge X_train and X_test
# X_train = np.vstack([X_train, X_test])
D = gower.gower_matrix(raw_df)


for linkage in ["average","complete","single"]:
    print(f"\n— linkage = {linkage} —")
    for k in range(2, 7):
        model  = AgglomerativeClustering(
                    n_clusters=k,
                    metric="precomputed",
                    linkage=linkage
                )
        labels = model.fit_predict(D)  # your Gower matrix
        sizes  = pd.Series(labels).value_counts().sort_index()
        sil    = silhouette_score(D, labels, metric="precomputed")
        print(f" k={k:>2} → sizes={sizes.to_dict()}  |  sil={sil:.3f}")


— linkage = average —
 k= 2 → sizes={0: 996, 1: 3}  |  sil=0.166
 k= 3 → sizes={0: 990, 1: 3, 2: 6}  |  sil=0.120
 k= 4 → sizes={0: 980, 1: 10, 2: 6, 3: 3}  |  sil=0.082
 k= 5 → sizes={0: 965, 1: 15, 2: 6, 3: 3, 4: 10}  |  sil=0.058
 k= 6 → sizes={0: 961, 1: 15, 2: 6, 3: 3, 4: 10, 5: 4}  |  sil=0.033

— linkage = complete —
 k= 2 → sizes={0: 854, 1: 145}  |  sil=0.120
 k= 3 → sizes={0: 485, 1: 145, 2: 369}  |  sil=0.019
 k= 4 → sizes={0: 369, 1: 145, 2: 288, 3: 197}  |  sil=0.028
 k= 5 → sizes={0: 145, 1: 197, 2: 288, 3: 217, 4: 152}  |  sil=0.021
 k= 6 → sizes={0: 288, 1: 197, 2: 80, 3: 217, 4: 152, 5: 65}  |  sil=0.018

— linkage = single —
 k= 2 → sizes={0: 998, 1: 1}  |  sil=0.223
 k= 3 → sizes={0: 997, 1: 1, 2: 1}  |  sil=0.160
 k= 4 → sizes={0: 996, 1: 1, 2: 1, 3: 1}  |  sil=0.103
 k= 5 → sizes={0: 995, 1: 1, 2: 1, 3: 1, 4: 1}  |  sil=0.068
 k= 6 → sizes={0: 994, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}  |  sil=0.041


Given your balance + silhouette trade-offs, complete linkage with k=4k=4k=4 is the sweet spot:
•	Cluster sizes
{369,  145,  288,  197} (all > 20)
•	Silhouette
0.028 on Gower distances (only slightly below the peak at k=2)
That gives you four reasonably large, interpretable segments—exactly in your 3–4 cluster sweet spot.



In [6]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import gower

# 1) raw_df: your pre-encoding DataFrame of 37 mixed‐type features
D = gower.gower_matrix(raw_df)

# 2) Fit with complete linkage and k=4
best_k = 4
model = AgglomerativeClustering(
    n_clusters=best_k,
    metric="precomputed",
    linkage="complete"
)
labels = model.fit_predict(D)

# 3) Confirm sizes & silhouette
sizes = pd.Series(labels).value_counts().sort_index()
sil   = silhouette_score(D, labels, metric="precomputed")
print(f"\nChosen k = {best_k}")
print(" Cluster sizes:", sizes.to_dict())
print(f" Silhouette (Gower) = {sil:.3f}\n")

# 4) (Optionally) inspect cluster means on raw_df for labeling
raw_df['Cluster'] = labels
# First identify numeric columns
numeric_cols = raw_df.select_dtypes(include=['number']).columns
# Then compute means
print(raw_df.groupby('Cluster')[numeric_cols].mean().iloc[:, :5].round(3))



Chosen k = 4
 Cluster sizes: {0: 369, 1: 145, 2: 288, 3: 197}
 Silhouette (Gower) = 0.028

          Month  Credit_Amount  Installment_Rate  Residence_Duration     Age
Cluster                                                                     
0        18.859       2728.309             3.005               2.694  34.859
1        26.745       4861.779             3.110               3.338  41.145
2        19.274       2855.771             2.712               2.694  31.694
3        22.888       3735.650             3.188               2.980  38.183


In [12]:
import numpy as np
import pandas as pd

# thresholds—tweak to taste
NUMERIC_STD_THR   = 0.05    # very low spread → almost constant
CATEGORICAL_PCT_THR = 0.90  # mode covers 90%+ → nearly unanimous

# separate cols
num_cols = raw_df.select_dtypes(include='number').columns.drop('Cluster')
cat_cols = raw_df.select_dtypes(include=['object','category']).columns

# precompute stats per cluster
grp = raw_df.groupby('Cluster')

# numeric: std per cluster
num_std = grp[num_cols].std()

# categorical: relative freq of the mode per cluster
def mode_pct(s):
    fq = s.value_counts(normalize=True)
    return fq.iloc[0]  if not fq.empty else 0
cat_mode_pct = grp[cat_cols].agg(mode_pct)

final_profiles = []
for cid in sorted(raw_df['Cluster'].unique()):
    imm = []
    # collect numeric‐immutables
    for f in num_cols:
        if num_std.loc[cid, f] < NUMERIC_STD_THR:
            imm.append(f)
    # collect categorical‐immutables
    for f in cat_cols:
        if cat_mode_pct.loc[cid, f] >= CATEGORICAL_PCT_THR:
            imm.append(f)

    # always include truly immutable by domain
    for global_imm in ["Age", "Foreign_Worker"]:  
        if global_imm not in imm:
            imm.append(global_imm)

    # actionable = all remaining features
    all_feats = num_cols.tolist() + cat_cols.tolist()
    act  = [f for f in all_feats if f not in imm]

    final_profiles.append({
      "cluster_id": cid,
      "immutable_features": sorted(imm),
      "actionable_features": sorted(act)
    })

# peek
for p in final_profiles:
    print(f"Cluster {p['cluster_id']}:")
    print("  Immutable:", p['immutable_features'])
    print("  Actionable:", p['actionable_features'][:5], "…(+", 
          len(p['actionable_features'])-5, "more)\n")


# def remove_circular(obj, seen=None):


# --- assume these are already computed from your raw_df + labels ---
# num_std:        DataFrame (clusters × numeric_cols) of within-cluster standard deviations
# cat_mode_pct:   DataFrame (clusters × cat_cols) of within-cluster mode frequency
# final_profiles: a list of dicts for each cluster, each with keys
#                 "cluster_id", "immutable_features", "actionable_features"

# thresholds
epsilon = 1e-6

# Build cost_weights for each profile
for prof in final_profiles:
    cid    = prof["cluster_id"]
    action = prof["actionable_features"]
    raw_w  = {}
    # numeric features: cost ∝ 1 / std   (harder to move → higher cost)
    for f in action:
        if f in num_std.columns:
            std = num_std.loc[cid, f]
            raw_w[f] = 1.0 / (std + epsilon)
        else:
            # categorical features: cost ∝ 1 / (1 - mode_pct)
            pct = cat_mode_pct.loc[cid, f]
            raw_w[f] = 1.0 / ((1.0 - pct) + epsilon)

    # normalize so average weight = 1.0 (sum weights = number of actionables)
    total = sum(raw_w.values())
    n_act = len(raw_w)
    cost_w = { f: (w * n_act / total) for f, w in raw_w.items() }

    prof["cost_weights"] = cost_w

# now peek at one

print(jsonpickle.encode(final_profiles))



clean_profiles = []
for prof in final_profiles:
    # Extract just built‐in types
    clean = {
        "cluster_id": prof["cluster_id"],
        "immutable_features": prof["immutable_features"],
        "actionable_features": prof["actionable_features"],
        "cost_weights": {f: float(w) for f, w in prof["cost_weights"].items()}
    }
    clean_profiles.append(clean)

with open("final_user_profiles_with_metadata.json", "w") as f:
    json.dump(clean_profiles, f, indent=2, default=lambda obj: obj.item() if hasattr(obj, 'item') else str(obj))



Cluster 0:
  Immutable: ['Age', 'Foreign_Worker', 'Other_Debtors', 'Other_Installment_Plans']
  Actionable: ['Credit_Amount', 'Credit_History', 'Employment', 'Existing_Credits', 'Housing'] …(+ 11 more)

Cluster 1:
  Immutable: ['Age', 'Foreign_Worker', 'Other_Debtors']
  Actionable: ['Credit_Amount', 'Credit_History', 'Employment', 'Existing_Credits', 'Housing'] …(+ 12 more)

Cluster 2:
  Immutable: ['Age', 'Foreign_Worker']
  Actionable: ['Credit_Amount', 'Credit_History', 'Employment', 'Existing_Credits', 'Housing'] …(+ 13 more)

Cluster 3:
  Immutable: ['Age', 'Foreign_Worker', 'Housing', 'Other_Debtors']
  Actionable: ['Credit_Amount', 'Credit_History', 'Employment', 'Existing_Credits', 'Installment_Rate'] …(+ 11 more)

[{"cluster_id": {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["i8", false, true]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}]}, {"py/b64": "AAAAAAAAAAA="}]}]}, "immuta

In [8]:


# # 1) Numeric summary
# num_cols = raw_df.select_dtypes(include='number').columns.drop('Cluster')
# cluster_num_means = (
#     raw_df
#     .groupby('Cluster')[num_cols]
#     .mean()
#     .round(3)
# )

# # 2) Categorical summary (mode)
# cat_cols = raw_df.select_dtypes(include=['object', 'category']).columns

# def mode_func(series):
#     m = series.mode()
#     return m.iloc[0] if not m.empty else None

# cluster_cat_modes = (
#     raw_df
#     .groupby('Cluster')[cat_cols]
#     .agg(mode_func)
# )

# # 3) Display
# print("=== Numeric feature means by cluster ===")
# print(cluster_num_means)

# print("\n=== Categorical feature modes by cluster ===")
# print(cluster_cat_modes)


=== Numeric feature means by cluster ===
          Month  Credit_Amount  Installment_Rate  Residence_Duration     Age  \
Cluster                                                                        
0        18.859       2728.309             3.005               2.694  34.859   
1        26.745       4861.779             3.110               3.338  41.145   
2        19.274       2855.771             2.712               2.694  31.694   
3        22.888       3735.650             3.188               2.980  38.183   

         Existing_Credits  Num_Liable_People  
Cluster                                       
0                   1.453              1.138  
1                   1.414              1.262  
2                   1.222              1.090  
3                   1.584              1.203  

=== Categorical feature modes by cluster ===
                      Status                            Credit_History  \
Cluster                                                                  
0 

In [9]:
# # 1. Summarize each cluster’s characteristics
# # Use the numeric means and categorical modes you just computed to write a 1–2 line “label” for each cluster.

# # numeric_means, categorical_modes from the last step
# for cid in range(4):
#     num = cluster_num_means.loc[cid]
#     cat = cluster_cat_modes.loc[cid]
#     # e.g. pick Purpose, Housing, Employment level from `cat`,
#     # Credit_Amount bucket and Age group from `num`, etc.
#     print(f"Cluster {cid}: credit≈{num.Credit_Amount:.0f}, age≈{num.Age:.0f}, "
#           f"housing={cat.Housing}, purpose={cat.Purpose}, …")


Cluster 0: credit≈2728, age≈35, housing=own, purpose=A43, …
Cluster 1: credit≈4862, age≈41, housing=for free, purpose=A40, …
Cluster 2: credit≈2856, age≈32, housing=own, purpose=A42, …
Cluster 3: credit≈3736, age≈38, housing=own, purpose=A43, …


In [20]:
# Load your JSON list of profiles
with open("final_user_profiles_with_metadata.json") as f:
    profiles = json.load(f)

for prof in profiles:
    # freeze‐list of encoded indices
    prof["freeze_idxs"] = []
    for raw in prof["immutable_features"]:
        prof["freeze_idxs"].extend(raw_to_encoded[raw])

    # build an encoded‐cost vector of length = n_encoded_cols
    cw = np.ones(len(encoded_cols), dtype=float)
    for raw, w in prof["cost_weights"].items():
        for idx in raw_to_encoded[raw]:
            cw[idx] = w
    prof["cost_vector"] = cw

    # sparsity λ is already prof["sparsity_lambda"]





In [ ]:
# --- Load Final Profile JSON ---
with open('final_user_profiles_with_metadata.json', 'r') as f:
    cluster_profiles = json.load(f)

# Build a lookup map
cluster_profile_map = {p['cluster_id']: p for p in cluster_profiles}

In [ ]:
# --- Clustering ---
k_fixed = len(cluster_profiles)
important_features = [33, 6, 28, 3, 23, 14, 4] # Mix immutable & actionable with High explanatory power These features had the highest SHAP/mutual-information scores in a quick feature-importance pass. Also They work with Euclidean linkage since they are Numeric/ordinal.

X_train_subset = X_train[:, important_features]
cluster_model = AgglomerativeClustering(n_clusters=k_fixed)
main_cluster_labels = cluster_model.fit_predict(X_train_subset)
print(f"Fixed number of clusters: {k_fixed}")

In [34]:
# --- Assign Test Samples to Clusters ---
centroid_model = NearestCentroid()
X_merge = np.vstack([X_train, X_test])
centroid_model.fit(X_merge, labels)
test_labels = centroid_model.predict(X_test)

results = []
for i, cid in enumerate(X_test):
    prof = profiles[test_labels[i]]
    gen  = load_model(f"generators/cluster_{prof['cluster_id']}.h5")

    cf = gen.predict(X_test[i:i+1])  # generate counterfactual for this sample
    cf = cf[0]  # remove batch dimension    
    

    # # enforce immutables
    # cf[prof["freeze_idxs"]] = X_test[prof["freeze_idxs"]]

    # evaluate / save
    results.append(cf)
    
def np_encoder(obj):
    import numpy as np
    if isinstance(obj, np.generic):
        return obj.item()
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

with open("your_output_file.json", "w") as f:
    json.dump(results, f, indent=2, default=np_encoder)

1/1 [==============================] - 0s 61ms/step


In [ ]:
# --- Prepare Cluster-Specific Training Data ---
cluster_data = defaultdict(lambda: {'X': [], 'y': []})
for x, y, cid in zip(X_train, y_train, main_cluster_labels):
    cluster_data[cid]['X'].append(x)
    cluster_data[cid]['y'].append(y)
for cid in cluster_data:
    cluster_data[cid]['X'] = np.vstack(cluster_data[cid]['X'])
    cluster_data[cid]['y'] = np.array(cluster_data[cid]['y'])

In [23]:
def train_all_cluster_gans(cluster_profiles,
                           cluster_data,
                           classifier,
                           output_dir='generators',
                           n_iterations=300):
    os.makedirs(output_dir, exist_ok=True)

    for prof in cluster_profiles:
        cid = prof['cluster_id']
        cw  = prof['cost_weights']
        lam = prof['sparsity_lambda']
        imm = prof['freeze_idxs']  # if you need it for weighted_version

        print(f"\n[Cluster {cid}] cost_weights = {cw}, sparsity_lambda = {lam}")

        # 1) Build fresh generator & discriminator
        gen  = create_generator(residuals=True)
        disc = create_discriminator()

        # 2) Define your CounterGAN (with sparsity loss)
        #    Pass cw dict and lambda into your define_countergan
        countergan = define_countergan(
            generator=gen,
            discriminator=disc,
            classifier=classifier,
            cost_weights=cw,
            sparsity_lambda= lam,
            immutable_idxs= imm  
        )

        # 3) Prepare the data stream
        Xc, yc = cluster_data[cid]['X'], cluster_data[cid]['y']
        batches = infinite_data_stream(Xc, yc, batch_size=128)

        # 4) Train
        t0 = time.time()
        # Note: train_countergan must now accept a prebuilt countergan model
        train_countergan(
            n_discriminator_steps=2,
            n_generator_steps=4,
            n_training_iterations=n_iterations,
            classifier=classifier,
            discriminator=disc,
            generator=gen,
            countergan=countergan,          # <-- pass it here
            batches=batches,
            weighted_version=True,
            # immutable_features=imm         # optionally if you do sample‐weighting
        )
        print(f"[Cluster {cid}] trained in {time.time()-t0:.1f}s")

        # 5) Save the generator for inference
        gen.save(f"{output_dir}/cluster_{cid}.h5")

In [24]:
# --- Train all personalized GANs per cluster ---
train_all_cluster_gans(
    cluster_profiles=prof,  
    cluster_data=cluster_data,
    classifier=classifier,
    output_dir='generators',
    n_iterations=300
)

NameError: name 'cluster_data' is not defined

In [ ]:
# --- Generate Personalized Counterfactuals ---
generators = {cid: load_model(f"generators/cluster_{cid}.h5")
              for cid in cluster_profile_map}

counterfactuals = []
personalized_latencies = []

for i, x0 in enumerate(X_test):
    cid = test_profiles[i]['cluster_id']
    gen = generators[cid]
    t0 = time.time()
    cf = gen.predict(x0[None])[0].astype(np.float32)
    personalized_latencies.append(1000*(time.time()-t0))
    counterfactuals.append(cf)

counterfactuals = np.stack(counterfactuals, axis=0)
latencies      = np.array(personalized_latencies, dtype=np.float32)

In [ ]:
# --- Overall Evaluation ---
from pprint import pprint

print("\nPersonalized GAN evaluation:")
personalized_metrics = compute_metrics(
    samples=X_test,
    counterfactuals=counterfactuals,
    latencies=latencies,
    classifier=classifier,
    autoencoder=autoencoder
)
pprint(personalized_metrics)

print("\nBaseline CounterGAN evaluation:")
baseline_cfs = generators[0].predict(X_test)
baseline_metrics = compute_metrics(
    samples=X_test,
    counterfactuals=baseline_cfs,
    latencies=np.zeros(len(X_test)),
    classifier=classifier,
    autoencoder=autoencoder
)
pprint(baseline_metrics)

In [ ]:
# --- Per-Cluster Metrics & Plots ---
cluster_metrics = []
for cid in np.unique(main_cluster_labels):
    idxs = [i for i,tp in enumerate(test_profiles) if tp['cluster_id']==cid]
    Xc_te = X_test[idxs]
    cf_c  = counterfactuals[idxs]
    lat_c = latencies[idxs]

    m = compute_metrics(
        samples=Xc_te,
        counterfactuals=cf_c,
        latencies=lat_c,
        classifier=classifier,
        autoencoder=autoencoder
    )
    m['cluster_id'] = cid
    m['size']       = len(idxs)
    cluster_metrics.append(m)

df_cluster = pd.DataFrame(cluster_metrics)
display(df_cluster[['cluster_id','size','prediction_gain','reconstruction_error','sparsity','latency']])

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,4))
ax1.bar(df_cluster['cluster_id'], df_cluster['sparsity'].str.split(' ± ').str[0].astype(float))
ax1.set_title('L1 distance by cluster')
ax2.bar(df_cluster['cluster_id'], df_cluster['prediction_gain'].str.split(' ± ').str[0].astype(float))
ax2.set_title('Prediction gain by cluster')
plt.show()

In [ ]:
import numpy as np

def auto_bump_cost_weights(cluster_profiles, cluster_data, generators, feature_names,
                           bump_factor=1.5, percentile_threshold=50):
    """
    Automatically bump cost_weights for actionable features in each cluster,
    based on per-cluster mean |delta| magnitudes.
    
    Parameters
    ----------
    cluster_profiles : list of dict
        Each dict contains 'cluster_id' and 'cost_weights' keys.
    cluster_data : dict
        Mapping cluster_id -> {'X': np.ndarray, ...}
    generators : dict
        Mapping cluster_id -> trained generator model
    feature_names : list[str]
        List of all feature names in X_train order.
    bump_factor : float
        Multiplicative factor to bump cost_weights by.
    percentile_threshold : float
        Delta threshold percentile (0-100). Features with mean delta above
        this percentile get bumped.
    """
    for prof in cluster_profiles:
        cid = prof['cluster_id']
        gen = generators[cid]
        Xc = cluster_data[cid]['X']  # shape (n_c, n_features)
        
        # Generate counterfactuals for the cluster
        cf = gen.predict(Xc)  # shape (n_c, n_features)
        
        # Compute mean absolute delta per feature
        mean_abs_delta = np.mean(np.abs(cf - Xc), axis=0)
        
        # Identify actionable features in this profile
        actionable = list(prof['cost_weights'].keys())
        actionable_idxs = [feature_names.index(f) for f in actionable]
        
        # Determine threshold based on percentile of actionable deltas
        actionable_deltas = mean_abs_delta[actionable_idxs]
        thresh = np.percentile(actionable_deltas, percentile_threshold)
        
        # Bump cost_weights for features exceeding the threshold
        for f, idx in zip(actionable, actionable_idxs):
            if mean_abs_delta[idx] > thresh:
                old_w = prof['cost_weights'][f]
                prof['cost_weights'][f] = old_w * bump_factor
                print(f"Cluster {cid}: bumped '{f}' weight {old_w:.2f} → {prof['cost_weights'][f]:.2f}")
                
    return cluster_profiles

# Use the correct feature names from your DataFrame
feature_names = features  # Already defined in your preprocessing function

# Print feature names to verify
print(f"Available features: {feature_names}")
print(f"Number of features: {len(feature_names)}")

# # Example usage:
# updated_profiles = auto_bump_cost_weights(
#     cluster_profiles=cluster_profiles,
#     cluster_data=cluster_data,
#     generators=generators,
#     feature_names=features,
#     bump_factor=1.5,
#     percentile_threshold=50
# )



In [ ]:
# 2) Check your semantic→index map for “Job”
print("Mapped ‘Job’ index →", feature_to_idx["Job"])
print("That corresponds to column:", feature_names[ feature_to_idx["Job"] ])


In [ ]:
# ❶ Freeze classifier once before anything else
classifier.trainable = False

# ❷ Define a finer lambda grid and longer fine-tune
to_tune = [0,4,5]
lambda_grid = [5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]
sweep_iters = 100

def eval_lambda_for_cluster(cid, lam):
    Xc, yc = cluster_data[cid]['X'], cluster_data[cid]['y']
    prof   = cluster_profile_map[cid]
    cw     = prof['cost_weights']
    
    # warm-start
    gen  = load_model(f"generators/cluster_{cid}.h5")
    disc = create_discriminator()
    cgan = define_countergan(gen, disc, classifier,
                             cost_weights=cw,
                             sparsity_lambda=lam)
    # fine-tune briefly
    batches = infinite_data_stream(Xc, yc, batch_size=128)
    train_countergan(2, 4, sweep_iters,
                     classifier=classifier,
                     discriminator=disc,
                     generator=gen,
                     countergan=cgan,
                     batches=batches,
                     weighted_version=True)
    # evaluate on that cluster’s test samples
    idxs = [i for i,tp in enumerate(test_profiles) if tp['cluster_id']==cid]
    Xc_te = X_test[idxs]
    cfs   = np.stack([gen.predict(x[None])[0] for x in Xc_te])
    m     = compute_metrics(Xc_te, cfs,
                            latencies=np.zeros(len(Xc_te)),
                            classifier=classifier,
                            autoencoder=autoencoder)
    gain = float(m['prediction_gain'].split(' ± ')[0])
    l1   = float(m['sparsity'].split(' ± ')[0])
    return gain, l1

results = {}
for cid in to_tune:
    rows = []
    for lam in lambda_grid:
        gain, l1 = eval_lambda_for_cluster(cid, lam)
        rows.append((lam, gain, l1))
    results[cid] = pd.DataFrame(rows, columns=['lambda','gain','l1'])
    print(f"\nCluster {cid} sweep:")
    display(results[cid])



In [ ]:
# ❶ Freeze classifier once before anything else
classifier.trainable = False

# ❷ Define a finer lambda grid and longer fine-tune
to_tune = [0,4,5]
lambda_grid = [5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]
sweep_iters = 100

def eval_lambda_for_cluster(cid, lam):
    Xc, yc = cluster_data[cid]['X'], cluster_data[cid]['y']
    prof   = cluster_profile_map[cid]
    cw     = prof['cost_weights']
    
    # warm-start
    gen  = load_model(f"generators/cluster_{cid}.h5")
    disc = create_discriminator()
    cgan = define_countergan(gen, disc, classifier,
                             cost_weights=cw,
                             sparsity_lambda=lam)
    # fine-tune briefly
    batches = infinite_data_stream(Xc, yc, batch_size=128)
    train_countergan(2, 4, sweep_iters,
                     classifier=classifier,
                     discriminator=disc,
                     generator=gen,
                     countergan=cgan,
                     batches=batches,
                     weighted_version=True)
    # evaluate on that cluster’s test samples
    idxs = [i for i,tp in enumerate(test_profiles) if tp['cluster_id']==cid]
    Xc_te = X_test[idxs]
    cfs   = np.stack([gen.predict(x[None])[0] for x in Xc_te])
    m     = compute_metrics(Xc_te, cfs,
                            latencies=np.zeros(len(Xc_te)),
                            classifier=classifier,
                            autoencoder=autoencoder)
    gain = float(m['prediction_gain'].split(' ± ')[0])
    l1   = float(m['sparsity'].split(' ± ')[0])
    return gain, l1

results = {}
for cid in to_tune:
    rows = []
    for lam in lambda_grid:
        gain, l1 = eval_lambda_for_cluster(cid, lam)
        rows.append((lam, gain, l1))
    results[cid] = pd.DataFrame(rows, columns=['lambda','gain','l1'])
    print(f"\nCluster {cid} sweep:")
    display(results[cid])



In [ ]:
# ❶ Freeze classifier once before anything else
classifier.trainable = False

# ❷ Define a finer lambda grid and longer fine-tune
to_tune = [6,7,8]
lambda_grid = [5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]
sweep_iters = 100

def eval_lambda_for_cluster(cid, lam):
    Xc, yc = cluster_data[cid]['X'], cluster_data[cid]['y']
    prof   = cluster_profile_map[cid]
    cw     = prof['cost_weights']
    
    # warm-start
    gen  = load_model(f"generators/cluster_{cid}.h5")
    disc = create_discriminator()
    cgan = define_countergan(gen, disc, classifier,
                             cost_weights=cw,
                             sparsity_lambda=lam)
    # fine-tune briefly
    batches = infinite_data_stream(Xc, yc, batch_size=128)
    train_countergan(2, 4, sweep_iters,
                     classifier=classifier,
                     discriminator=disc,
                     generator=gen,
                     countergan=cgan,
                     batches=batches,
                     weighted_version=True)
    # evaluate on that cluster’s test samples
    idxs = [i for i,tp in enumerate(test_profiles) if tp['cluster_id']==cid]
    Xc_te = X_test[idxs]
    cfs   = np.stack([gen.predict(x[None])[0] for x in Xc_te])
    m     = compute_metrics(Xc_te, cfs,
                            latencies=np.zeros(len(Xc_te)),
                            classifier=classifier,
                            autoencoder=autoencoder)
    gain = float(m['prediction_gain'].split(' ± ')[0])
    l1   = float(m['sparsity'].split(' ± ')[0])
    return gain, l1

# results = {}
# for cid in to_tune:
#     rows = []
#     for lam in lambda_grid:
#         gain, l1 = eval_lambda_for_cluster(cid, lam)
#         rows.append((lam, gain, l1))
#     results[cid] = pd.DataFrame(rows, columns=['lambda','gain','l1'])
#     print(f"\nCluster {cid} sweep:")
#     display(results[cid])



In [ ]:
# 1) Identify clusters with L1 > 6.0
high_l1 = df_cluster[
    df_cluster['sparsity'].str.split(' ± ').str[0].astype(float) > 6.0
]['cluster_id'].tolist()
print("Clusters to retune λ:", high_l1)

# 2) New λ grid (higher values)
lambda_grid = [5e-5, 1e-4, 5e-4, 1e-3, 5e-3]
sweep_iters = 100

# 3) Re-use your eval_lambda_for_cluster
results = {}
for cid in high_l1:
    rows = []
    for lam in lambda_grid:
        gain, l1 = eval_lambda_for_cluster(cid, lam)
        rows.append((lam, gain, l1))
    results[cid] = pd.DataFrame(rows, columns=['lambda','gain','l1'])
    print(f"\nCluster {cid} λ-sweep:")
    display(results[cid])

# 4) Pick for each cluster the smallest λ that brings l1 below your target
best_lams = {}
for cid, df in results.items():
    cand = df[df['l1'] <= 6.0]
    if not cand.empty:
        best = cand.sort_values('lambda').iloc[0]['lambda']
    else:
        # fallback to λ that minimizes l1 (even if >6)
        best = df.sort_values('l1').iloc[0]['lambda']
    best_lams[cid] = best

print("Chosen λ per cluster:", best_lams)

# 5) Inject into your JSON
with open("final_user_profiles_with_metadata.json","r") as f:
    profiles = json.load(f)
for p in profiles:
    cid = p['cluster_id']
    if cid in best_lams:
        p['sparsity_lambda'] = float(best_lams[cid])
with open("final_user_profiles_with_metadata.json","w") as f:
    json.dump(profiles, f, indent=2)
